# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using surprise. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives
You will be able to:
* Fit a recommender system model to a set of data
* Create a function that will return the top recommendations for a user
* Introduce a new user to a rating matrix and make recommendations for them

For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last  lesson, you got exposed to working with Surprise datasets. In this lab, you will also go through the process of reading in a dataset into the Surprise dataset format. To begin with, load the dataset into a pandas dataframe. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [1]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
userId       100836 non-null int64
movieId      100836 non-null int64
rating       100836 non-null float64
timestamp    100836 non-null int64
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [2]:
#drop unnecessary columns
new_df= df.drop(columns = 'timestamp')

It's now time to transform the dataset into something compatible with Surprise. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [5]:
from surprise import Reader, Dataset
# read in values as Surprise dataset 
reader = Reader()
data = Dataset.load_from_df(new_df, reader)

Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [6]:
dataset = data.build_full_trainset()
print('Number of users: ',dataset.n_users,'\n')
print('Number of items: ',dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the Best Model
Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [7]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [9]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run
params = {'n_factors': [20, 50, 100],
         'reg_all': [0.02, 0.05, 0.1]}
grid_svd = GridSearchCV(SVD, param_grid = params, n_jobs = -1)
grid_svd.fit(data)

In [10]:
# print out optimal parameters for SVD after GridSearch
print(grid_svd.best_score)
print(grid_svd.best_params)

{'rmse': 0.8690640118873543, 'mae': 0.6680769281422995}
{'rmse': {'n_factors': 100, 'reg_all': 0.05}, 'mae': {'n_factors': 100, 'reg_all': 0.05}}


In [11]:
# cross validating with KNNBasic
basic = KNNBasic(sim_options = {'name': 'pearson', 'user_based': True})
cv_basic = cross_validate(basic, data, n_jobs = -1)

In [14]:
# print out the average RMSE score for the test set
for i in cv_basic.items():
    print(i)
    print('--------------------------------')
    print(np.mean(cv_basic['test_rmse']))

('test_rmse', array([0.96876367, 0.97865332, 0.96719704, 0.96633094, 0.98145412]))
--------------------------------
0.9724798172111935
('test_mae', array([0.74872162, 0.75290865, 0.74594999, 0.74881002, 0.75689038]))
--------------------------------
0.9724798172111935
('fit_time', (1.4620463848114014, 1.5119538307189941, 1.4910178184509277, 1.485029935836792, 1.7094244956970215))
--------------------------------
0.9724798172111935
('test_time', (2.839402437210083, 2.7405076026916504, 3.0438146591186523, 3.290116310119629, 2.666245698928833))
--------------------------------
0.9724798172111935


In [15]:
# cross validating with KNNBaseline
knn_base = KNNBaseline(sim_options = {'name': 'pearson', 'user_based': True})
cv_base = cross_validate(knn_base, data, n_jobs = -1)

In [16]:
# print out the average score for the test set
for i in cv_base.items():
    print(i)
    print('-----------------')
    print(np.mean(cv_base['test_rmse']))

('test_rmse', array([0.87714458, 0.87508216, 0.88005022, 0.87384892, 0.87468733]))
-----------------
0.8761626442215407
('test_mae', array([0.6684324 , 0.6696106 , 0.67441309, 0.66694086, 0.66740901]))
-----------------
0.8761626442215407
('fit_time', (1.5847594738006592, 1.5933043956756592, 1.6007168292999268, 1.6017124652862549, 1.6140375137329102))
-----------------
0.8761626442215407
('test_time', (3.9375061988830566, 3.9614033699035645, 3.869645357131958, 4.075458288192749, 3.617035150527954))
-----------------
0.8761626442215407


Based off these outputs, it seems like the best performing model is the SVD model with n_factors = 50 and a regularization rate of 0.05. Let's use that model to make some predictions. Use that model or if you found one that performs better, feel free to use that.

## Making Recommendations

This next section is going to involve making recommendations, and it's important that the output for the recommendation is interpretable to people. Rather than returning the movie_id values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [17]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [18]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [19]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [20]:
svd.predict(2,4)

Prediction(uid=2, iid=4, r_ui=None, est=3.2027191636971724, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put all of our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step to go let's create a function that allows students to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater` should take as parameters:
* movie_df : DataFrame - a dataframe containing the movie ids, name of movie, and genres
* num : int - number of ratings
* genre : string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int  , 'movieId': int  ,'rating': float  }

#### This function is optional, but fun :) 

In [25]:
def movie_rater(movie_df,num, genre=None):
    userID = 1000
    rating_list = []
    while num > 0:
        if genre:
            movie = movie_df[movie_df['genres'].str.contains(genre)].sample(1)
        else:
            movie = movie_df.sample(1)
        print(movie)
        rating = input('How do you rate this movie on a scale of 1-5, press n if you have not seen :\n')
        if rating == 'n':
            continue
        else:
            rating_one_movie = {'userId':userID,'movieId':movie['movieId'].values[0],'rating':rating}
            rating_list.append(rating_one_movie) 
            num -= 1
    return rating_list

In [28]:
# try out the new function here!
user_rating = movie_rater(df_movies, 4, 'Comedy')
user_rating

      movieId              title        genres
9718   189043  Boundaries (2018)  Comedy|Drama
How do you rate this movie on a scale of 1-5, press n if you have not seen :
n
      movieId             title            genres
9051   141844  12 Chairs (1971)  Adventure|Comedy
How do you rate this movie on a scale of 1-5, press n if you have not seen :
n
      movieId               title                genres
1163     1542  Brassed Off (1996)  Comedy|Drama|Romance
How do you rate this movie on a scale of 1-5, press n if you have not seen :
n
      movieId                          title  genres
9319   159849  Bo Burnham: Make Happy (2016)  Comedy
How do you rate this movie on a scale of 1-5, press n if you have not seen :
n
      movieId                  title           genres
7531    84696  Burke and Hare (2010)  Comedy|Thriller
How do you rate this movie on a scale of 1-5, press n if you have not seen :
n
     movieId                title                        genres
696      914  My Fair

[{'userId': 1000, 'movieId': 914, 'rating': '4'},
 {'userId': 1000, 'movieId': 67087, 'rating': '3'},
 {'userId': 1000, 'movieId': 2369, 'rating': '3'},
 {'userId': 1000, 'movieId': 134170, 'rating': '2'}]

If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [29]:
user_rating

[{'userId': 1000, 'movieId': 914, 'rating': '4'},
 {'userId': 1000, 'movieId': 67087, 'rating': '3'},
 {'userId': 1000, 'movieId': 2369, 'rating': '3'},
 {'userId': 1000, 'movieId': 134170, 'rating': '2'}]

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a Surprise dataset
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number)

In [30]:
## add the new ratings to the original ratings DataFrame
new_ratings_df = new_df.append(user_rating, ignore_index = True)
new_data = Dataset.load_from_df(new_ratings_df, reader)

In [32]:
# train a model using the new combined DataFrame
svd = SVD(n_factors = 50, reg_all = 0.05)
svd.fit(new_data.build_full_trainset())

In [36]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
list_of_movies = []
for m_id in new_df['movieId'].unique():
    list_of_movies.append((m_id, svd.predict(1000, m_id)[3]))

In [39]:
# order the predictions from highest to lowest rated

ranked_movies = sorted(list_of_movies, key = lambda x: x[1], reverse = True)
ranked_movies

[(750, 4.2452400719991905),
 (58559, 4.141814864141856),
 (48516, 4.118753427035098),
 (2959, 4.118634874730576),
 (1248, 4.118126036377155),
 (318, 4.115265691434601),
 (1178, 4.113252272740624),
 (5690, 4.09904882942039),
 (50, 4.097062002101988),
 (1199, 4.090805826038206),
 (1198, 4.0884945843332385),
 (1104, 4.088343134941577),
 (866, 4.085955749390869),
 (296, 4.079899128711497),
 (5618, 4.072609276363611),
 (3275, 4.067309523336404),
 (1204, 4.064403498413035),
 (720, 4.056034441494206),
 (858, 4.053977643844345),
 (1233, 4.053256687393413),
 (1261, 4.051867101625212),
 (6711, 4.047962402499436),
 (1276, 4.043599837494142),
 (56782, 4.042695926135796),
 (1250, 4.042647444711202),
 (356, 4.041382215366022),
 (38061, 4.040934989063165),
 (1196, 4.037778246476025),
 (933, 4.032905915406235),
 (3508, 4.029949797586381),
 (475, 4.029796688616529),
 (1089, 4.027852382335744),
 (2329, 4.0247956370598805),
 (1213, 4.0213145721406365),
 (1262, 4.018572966142345),
 (1136, 4.01755464408004

 For the final component of this challenge, it could be useful to create a function `recommended_movies` that takes in the parameters:
* `user_ratings` : list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df` : DataFrame 
* `n` : int- number of recommended movies 

The function should use a for loop to print out each recommended *n* movies in order from best to worst

In [41]:
# return the top n recommendations using the 
def recommended_movies(user_ratings,movie_title_df,n):
        for idx, rec in enumerate(user_ratings):
            title = movie_title_df.loc[movie_title_df['movieId'] == int(rec[0])]['title']
            print('Recommendation # ',idx+1,': ',title,'\n')
            n-= 1
            if n == 0:
                break
            
recommended_movies(ranked_movies,df_movies,5)

Recommendation #  1 :  602    Dr. Strangelove or: How I Learned to Stop Worr...
Name: title, dtype: object 

Recommendation #  2 :  6710    Dark Knight, The (2008)
Name: title, dtype: object 

Recommendation #  3 :  6315    Departed, The (2006)
Name: title, dtype: object 

Recommendation #  4 :  2226    Fight Club (1999)
Name: title, dtype: object 

Recommendation #  5 :  947    Touch of Evil (1958)
Name: title, dtype: object 



## Level Up

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top n recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the change to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will get exposed to using spark to make recommender systems.